# Making the ship function

Before we start to code for piloting the ship, making it fire bullets etc. we will learn about a concept called called refactoring which will help in organizing/ managing your code better.

## Refactoring : the *game_functions* module
In larger projects, you’ll often refactor code the you’ve written before adding more. Refactoring simplifies the structure of the code you’ve already written, making it easier to build on. In this section we’ll create a new module called **game_functions**, which will store a number of functions that make Alien Invasion work. The game_functions module will prevent *alien_invasion.py* from becoming too lengthy and will make the logic in *alien_invasion.py* easier to follow.


To simplify *run_game* function we will first make 2 functions in the game_functions module: `check_events()` and `update_screen`.  

Make the file game_functions.py with the following code.

In [ ]:
import sys
import pygame

def check_events():
  """Respond to keypresses and mouse events."""
  for event in pygame.event.get():
    if event.type == pygame.QUIT:
      sys.exit()

def update_screen(bg_img, screen, ship):
  """Update images on the screen and flip to the new screen."""
  
  # Redraw the screen during each pass through the loop.
  screen.blit(bg_img, (0,0))
  ship.blitme()

  # Make the most recently drawn screen visible.
  pygame.display.flip()



We simply moved the code which managed the events to this separted `check_events` function. Isolating the event loop in such a way, allows you to manage events separately from other aspects of the game.

`update_screen()` function takes three parameters: bg_img,
screen, and ship and simply redraws the screen.

Using this here is how *alien_inavsion.py* will be modified.

In [ ]:
import pygame
from settings import Settings
from ship import Ship
import game_functions as gf

def run_game():
    # Initialize pygame, settings, and screen object.
    pygame.init()

    ai_settings = Settings()

    screen = pygame.display.set_mode((ai_settings.screen_width, ai_settings.screen_height))
    pygame.display.set_caption("Alien Invasion")

    # Make a ship.
    ship = Ship(screen)

    #Load the background image
    bg_img = pygame.image.load('images/bg.jpg')

    # Start the main loop for the game.
    while True:
     gf.check_events()
     gf.update_screen(bg_img, screen, ship)

run_game()

You can see now how such refactoring simplified our code in alien_invasion.py.  
*Also note we no longer need to import sys directly into the main program file, because it’s only being used in the game_functions module now.*

## Piloting the ship
Let’s give the player the ability to move the ship right and left. To do this, we’ll write code that responds when the player presses the right or left arrow key. We’ll focus on movement to the right first, and then we’ll apply the same principles to control movement to the left. As you do this, you’ll learn how to control the movement of images on the screen.


Whenever the player presses a key, that keypress is registered in Pygame as an event. Each event is picked up by the `pygame.event.get()` method, so we need to specify in our check_events() function what kind of events to check for. Each keypress is registered as a `KEYDOWN` event. When a `KEYDOWN` event is detected, we need to check whether the key that was pressed is one that triggers a certain event.  
Here to detect pressing the right arrow key we can add the code directly in the `check_events()` function in *game_functions.py*.
We can modify it in the following manner:

In [ ]:
import sys
import pygame

def check_events(ship):
  """Respond to keypresses and mouse events."""
  for event in pygame.event.get():
    if event.type == pygame.QUIT:
      sys.exit()
  elif event.type == pygame.KEYDOWN:
    if event.key == pygame.K_RIGHT:
      # Move the ship to the right.
      ship.rect.centerx += 1


def update_screen(bg_img, screen, ship):
  """Update images on the screen and flip to the new screen."""
  
  # Redraw the screen during each pass through the loop.
  screen.blit(bg_img, (0,0))
  ship.blitme()

  # Make the most recently drawn screen visible.
  pygame.display.flip()

We give the `check_events()` function a ship parameter, because the ship needs to move to the right when the right arrow key is pressed. Inside `check_events()` we add an *elif* block to the event loop to respond when Pygame detects a `KEYDOWN` event. We check if the key pressed is the right arrow key (`pygame.K_RIGHT`) by reading the `event.key` attribute. If the right arrow key was pressed, we move the ship to the right by increasing the value of `ship.rect.centerx` by 1.

While calling the function in *alien_invasion.py* we also need to pass *ship* object as an arguement.



In [ ]:
# Start the main loop for the game.
while True:
  gf.check_events(ship)
  gf.update_screen(bg_img, screen, ship)


### Allowing Continuous Movement
When the player holds down the right arrow key, we want the ship to
continue moving right until the player releases the key. We’ll have our game detect a `pygame.KEYUP` event so we’ll know when the right arrow key is released; then we’ll use the `KEYDOWN` and `KEYUP` events together with a flag called `moving_right` to implement continuous motion.

When the ship is motionless, the moving_right flag will be **False**. When the right arrow key is pressed, we’ll set the flag to **True**, and when it’s released, we’ll set the flag to **False** again.

The Ship class controls all attributes of the ship, so we’ll give it an attribute called `moving_right` and an `update()` method to check the status of the `moving_right` flag.

Here's how the ship class in *ship.py* will be modified.

In [ ]:
class Ship():

  def __init__(self, screen):
    """Initialize the ship and set its starting position."""

    self.screen = screen #This refers to the screen object we had created
    
    # Load the ship image and get its rect.
    self.image = pygame.image.load('images/ship.png')
    self.rect = self.image.get_rect()
    self.screen_rect = screen.get_rect()

    # Start each new ship at the bottom center of the screen.
    self.rect.centerx = self.screen_rect.centerx
    self.rect.bottom = self.screen_rect.bottom

    # Movement flag
    self.moving_right = False

  def update(self):
    """Update the ship's position based on the movement flag."""
    if self.moving_right:
      self.rect.centerx += 1

  def blitme(self):
    --snip--


We add a self.moving_right attribute in the `__init__()` method and set it to **False** initially. Then we add `update()`, which moves the ship right if the flag is **True**. 

Now modify `check_events()` so that moving_right is set to True when the right arrow key is pressed and False when the key is released.

Here's how it will be modified in *game_functions.py* will be modified :

In [ ]:
def check_events(ship):
  """Respond to keypresses and mouse events."""
  for event in pygame.event.get():
    --snip--
    elif event.type == pygame.KEYDOWN:
      if event.key == pygame.K_RIGHT:  
        ship.moving_right = True  
    elif event.type == pygame.KEYUP:
      if event.key == pygame.K_RIGHT:
        ship.moving_right = False


We modify how the game responds when the player presses the
right arrow key: instead of changing the ship’s position directly, we merely set `moving_right` to True. We then add a new elif block, which responds to `KEYUP` events. When the player releases the right arrow key (K_RIGHT), we set `moving_right` to **False**.


Finally, we modify the while loop in alien_invasion.py so it calls the ship’s update() method on each pass through the loop:


In [ ]:
  # Start the main loop for the game.
  while True:
    gf.check_events(ship)
    ship.update()
    gf.update_screen(bg_img, screen, ship)


The ship’s position will update after we’ve checked for keyboard
events and before we update the screen.

### Moving both left and right

We’ll have to modify the Ship class and the check_events() function in a similar manner. Here are the relevant changes to `__init__()` and `update()` in Ship:


In [ ]:
class Ship():

  def __init__(self, screen):
    """Initialize the ship and set its starting position."""

    self.screen = screen #This refers to the screen object we had created
    
    # Load the ship image and get its rect.
    self.image = pygame.image.load('images/ship.png')
    self.rect = self.image.get_rect()
    self.screen_rect = screen.get_rect()

    # Start each new ship at the bottom center of the screen.
    self.rect.centerx = self.screen_rect.centerx
    self.rect.bottom = self.screen_rect.bottom

    # Movement flag
    self.moving_right = False
    self.moving_left = False

  def update(self):
    """Update the ship's position based on the movement flag."""
    if self.moving_right:
      self.rect.centerx += 1
    if self.moving_left:
      self.rect.centerx -= 1

  def blitme(self):
    --snip--

We have to make 2 modifcations to `check_events()` in *game_functions.py* as well.

In [ ]:
def check_events(ship):
  """Respond to keypresses and mouse events."""
  for event in pygame.event.get():
    --snip--
    elif event.type == pygame.KEYDOWN:
      if event.key == pygame.K_RIGHT:  
        ship.moving_right = True 
      elif event.key == pygame.K_LEFT:  
        ship.moving_left = True 

    elif event.type == pygame.KEYUP:
      if event.key == pygame.K_RIGHT:
        ship.moving_right = False
      elif event.key == pygame.K_LEFT:
        ship.moving_left = False

At this point however the ship will disappear off either edge of the screen if you hold down an arrow key long enough. Let’s correct this so the ship stops moving when it reaches the edge of the screen. We do this by modifying the update() method in Ship:


In [ ]:
def update(self):
  """Update the ship's position based on movement flags."""
  if self.moving_right and self.rect.right < self.screen_rect.right:
    self.rect.centerx += 1
  if self.moving_left and self.rect.left > 0:
    self.rect.centerx -= 1


Before moving on we can refactor the `check_events()` making it easier to understand and organize the code there. Thus, we add 2 more functions in *game_functions.py*


In [ ]:
import sys
import pygame

def check_events(ship):
  """Respond to keypresses and mouse events."""
  for event in pygame.event.get():
    if event.type == pygame.QUIT:
      sys.exit()
    elif event.type == pygame.KEYDOWN:
      check_keydown_events(event,ship) 

    elif event.type == pygame.KEYUP:
      check_keyup_events(event,ship)

def check_keydown_events(event, ship):
  """respond to keypresses"""
  if event.key == pygame.K_RIGHT:  
    ship.moving_right = True 
  elif event.key == pygame.K_LEFT:  
    ship.moving_left = True

def check_keyup_events(event, ship):
  """ respond to key releases"""
  if event.key == pygame.K_RIGHT:
    ship.moving_right = False
  elif event.key == pygame.K_LEFT:
    ship.moving_left = False

def update_screen(bg_img, screen, ship):
  """Update images on the screen and flip to the new screen."""
  
  # Redraw the screen during each pass through the loop.
  screen.blit(bg_img,(0,0))
  ship.blitme()

  # Make the most recently drawn screen visible.
  pygame.display.flip()


We make two new functions: `check_keydown_events()` and `check_keyup_
events()`. Each needs an event parameter and a ship parameter. The bodies
of these two functions are copied from `check_events()`, and we’ve replaced
the old code with calls to the new functions.